<a href="https://colab.research.google.com/github/Valtern/PCVK_Ganjil_2025/blob/main/week_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import google.colab.drive

google.colab.drive.mount('/content/drive')

import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import math
from skimage import io
import os
import glob
from google.colab.patches import cv2_imshow
from PIL import Image as im

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Part A

1. Tampilkan histogram tiap citra dan analisis distribusi intensitasnya

In [ ]:
img1 = cv.imread('')

2. Terapkan transformasi brightness dan contrast (linear/log brightness).

o Tentukan nilai b (brightness) dan a (contrast) yang sesuai agar wajah tampak
natural

3. Lakukan histogram equalization untuk memperbaiki sebaran kontras.

o Bandingkan hasil visual dan histogram sebelum–sesudah.

4.  Terapkan filter spasial:

o Low-pass filter untuk menghaluskan noise kulit wajah.

o High-pass atau Laplacian filter untuk menajamkan tepi mata dan bibir

5. Implementasikan Floyd–Steinberg Dithering untuk menurunkan kedalaman warna
wajah (bit-depth 4–6 bit), lalu analisis bagaimana efeknya terhadap detail dan ekspresi
wajah.